In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import random
import math

In [122]:
df = pd.read_csv('housing.csv', header=None, sep='\s+')  # sep='\s+' or  sep='\t' or
column_names = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
    'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV'
]


df.columns = column_names
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\akaku\AppData\Local\Temp\ipykernel_4644\743828371.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('housing.csv', header=None, sep='\s+')  # sep='\s+' or  sep='\t' or


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [123]:
X = df.drop(df.columns[-1], axis=1).to_numpy()
y = df['MEDV'].to_numpy()

In [124]:
test_size = 0.25

row_count = X.shape[0] 
row_count_in_samples = int(row_count * test_size)

indixes = np.random.permutation(row_count)

test_indices = indixes[:row_count_in_samples]
train_indices = indixes[row_count_in_samples:]


X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

Linear Regression 

In [ ]:
def linear_regression(
        X: np.ndarray,
        y: np.ndarray,
        learning_rate: float = 0.01,
        iterations: int = 5000,
        coef_umensheniya_learning_rate: float = 0.001,
        epsilon: float = 1e-4
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Линейная регрессия.

    :param X: Исходная матрица.
    :param y: Целевые значения.
    :param learning_rate: Начальная скорость.
    :param iterations: максимальное количество итераций.
    :param coef_umensheniya_learning_rate: Коэффициент уменьшения скорости сходимости.
    :param epsilon: показатель качества модели для досрочного выхода из обучения.
    :return: Кортеж: 
        - weights: Веса.
        - X_mean: Средние значения.
        - X_std: Стандартные отклонения.
    """
    # нормализация признаков
    X_mean = np.mean(X, axis=0 )
    X_std = np.std(X, axis=0)
    # Избегаем деления на ноль
    X_std[X_std == 0] = 1
    X_normalized = (X - X_mean) / X_std
    
    # Добавление столбца единиц для свободного члена
    row, col = X_normalized.shape
    X_b = np.column_stack([np.ones(row), X_normalized])

    weights = np.random.randn(col + 1)

    for i in range(iterations):
        y_pred = X_b @ weights
        error = y_pred - y
        error_absolute = np.sign(error) # этот вариант сходится в 2 раза медленнее 5000 iter vs 10000 iter

        gradient = (2 / row) * X_b.T @ error
        current_lr = learning_rate * (1 / (1 + coef_umensheniya_learning_rate * i))
        old_weights = weights.copy()
        weights = weights - current_lr * gradient

        # Проверяем изменение весов
        weight_change = np.linalg.norm(weights - old_weights)
        
        if weight_change < epsilon:
            print(f"Сходимость достигнута на итерации {i}, изменение весов: {weight_change:.2e}")
            break

        if i == iterations - 1:
            print(f"Заданная точность НЕ достигнута за {iterations} шагов, последнее изменение весов: {weight_change:.2e}")
    
    return weights, X_mean, X_std

def predict(X, weights, X_mean, X_std):
    """
    Предсказание.
    
    :param X: Исходная матрица.
    :param weights: Веса.
    :param X_mean:Средние значения.
    :param X_std: Стандартные отклонения.
    :return: Предсказание.
    """
    X_normalized = (X - X_mean) / X_std
    row = X_normalized.shape[0]
    X_b = np.column_stack([np.ones(row), X_normalized])
    return X_b @ weights

In [126]:
weights, X_mean, X_std = linear_regression(X_train, y_train, iterations=20000)


y_predict = predict(X_test, weights, X_mean, X_std)

Сходимость достигнута на итерации 5124, изменение весов: 1.00e-04


sklearn

In [127]:
regression = LinearRegression()
regression.fit(X_train, y_train)
y_predict_sk = regression.predict(X_test)

Результаты

In [128]:
for i in range(len(y_predict)):
    print(f'{y_predict[i]}\t{y_predict_sk[i]}\t{y_test[i]}')

22.92955473846173	22.956832850081717	22.4
34.834818920187274	34.620471943870896	34.9
32.213310115356144	32.107005379959936	29.0
20.62412386787098	21.205397494426347	18.8
27.089331825457766	27.389603761312205	20.6
32.80841549112212	33.00746161769821	28.2
19.332554911388844	19.351523367893993	22.6
15.06195405612306	15.638614555904052	15.7
23.697375770910604	23.72196455845748	20.3
20.073311087340812	20.02089982188134	16.8
20.87556168715093	20.844159923280696	21.5
24.9920014632119	25.067077095097726	22.7
22.367630141160774	21.812686677026747	16.5
25.04358029224152	25.120167492841173	24.7
13.627748410779517	13.60781880651551	15.0
11.9028389536087	11.971652508347741	12.0
21.011402689965696	21.10598955843161	20.0
25.682474981710726	25.76463580503949	24.0
26.138806666110945	26.149357657463838	22.2
11.768891262896947	11.72140109370325	12.7
21.844707956149545	21.899170968364572	21.4
29.183124881730972	29.4704801220583	25.0
14.23613700162841	14.210929910814947	9.6
19.941474595812828	19.9685808083